Cleaning and analyzing dataset of used cars from eBay Kleinanzeigend

In this guided project, we'll work with a dataset of used cars from eBay Kleinanzeigen, a classifieds section of the German eBay website.

The dataset was originally scraped and uploaded to Kaggle. We've made a few modifications from the original dataset that was uploaded to Kaggle

In [61]:
import numpy as ny
import pandas as pd

autos = pd.read_csv("autos.csv", encoding='Latin-1')


In [62]:
autos.info()
# There are 20 columns, 5 with integers and rest with strings
# The column names use camelcase instead of Python's preferred snakecase, which means we can't just replace spaces with underscores
# time comes in format 00:00:00
# Some columns have null values, but none have more than ~20% null values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

In [63]:
autos.head()


,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


Now we will convert the column names from camelcase to snakecase and reword some of the column names based on the data dictionary to be more descriptive

In [64]:
cols = autos.columns
print(cols)

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')


In [65]:
autos.columns = (cols
                 .str.replace('yearOfRegistration','registration_year')
                 .str.replace('monthOfRegistration','registration_month')
                 .str.replace('notRepairedDamage','unrepaired_damage')
                 .str.replace('dateCreated','ad_created')
                 .str.replace('nrOfPictures','num_photos')
                 .str.replace('offerType', 'offer_type')
                 .str.replace('vehicleType', 'vehicle_type')
                 .str.replace('fuelType', 'fuel_type')
                 .str.replace('dateCrawled', 'date_crawled')
                 .str.replace('lastSeen', 'last_seen')
                )

In [66]:
autos.head()

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,powerPS,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_photos,postalCode,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In [67]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,powerPS,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_photos,postalCode,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-30 19:48:02,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


Columns seller and offerType are almost all of the same value, they should be dropped
odometer and price are numeric data stored as string
num_photos has all 0 in descriptive stats, need more investigation

In [68]:
autos['price'] = (autos["price"]
                        .str.replace('$','')
                        .str.replace(',','')
                        .astype(int))


In [69]:
autos['odometer'] = (autos["odometer"]
                    .str.replace('km','')
                    .str.replace(',','')
                     .astype(int)
                    )

In [70]:
autos.rename(columns = {'odometer':'odometer_km'}, inplace=True)

In [71]:
autos.num_photos.value_counts(dropna=False) #num_photos also has all the same values, should drop
autos = autos.drop(['seller','offer_type','num_photos'],axis=1)

In [72]:
autos['odometer_km'].unique().shape # there are 13 unique numbers

(13,)

In [73]:
autos['odometer_km'].describe()

count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

In [74]:
autos['odometer_km'].value_counts().sort_index(ascending=True)
# most of the cars have 150000 km but there is no outliers

5000        967
10000       264
20000       784
30000       789
40000       819
50000      1027
60000      1164
70000      1230
80000      1436
90000      1757
100000     2169
125000     5170
150000    32424
Name: odometer_km, dtype: int64

In [75]:
autos['price'].unique().shape #2357 prices
autos['price'].describe() # minimum is 0 so there must be a mistake
autos['price'].value_counts().sort_index(ascending=True)
# After 350,000 , the prices suddenly jump by 3 times to 999990, 
# this and prices above look like outliers.
# We are also going to remove 0 prices
autos = autos[autos['price'].between(1,350000)]

In [76]:
# Different columns are representing dates
# those are: date_crawled, last_seen, ad_created, registration montj
# and registration_year
# some dates are stored as string instead of numbers

In [77]:
# We are going to calculate the distribution of claues in the
# dates stored as strings

In [78]:
autos[['date_crawled','last_seen',
      'ad_created','registration_month',
      'registration_year']].info()

autos[['date_crawled','last_seen',
      'ad_created']][0:5]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48565 entries, 0 to 49999
Data columns (total 5 columns):
date_crawled          48565 non-null object
last_seen             48565 non-null object
ad_created            48565 non-null object
registration_month    48565 non-null int64
registration_year     48565 non-null int64
dtypes: int64(2), object(3)
memory usage: 2.2+ MB


,date_crawled,last_seen,ad_created
0,2016-03-26 17:47:46,2016-04-06 06:45:54,2016-03-26 00:00:00
1,2016-04-04 13:38:56,2016-04-06 14:45:08,2016-04-04 00:00:00
2,2016-03-26 18:57:24,2016-04-06 20:15:37,2016-03-26 00:00:00
3,2016-03-12 16:58:10,2016-03-15 03:16:28,2016-03-12 00:00:00
4,2016-04-01 14:38:50,2016-04-01 14:38:50,2016-04-01 00:00:00


In [79]:
(autos['date_crawled'].str[:10].
 value_counts(normalize=True, dropna=False).
 sort_index())

# Just separating the date
# most of the dates are in April

2016-03-05    0.025327
2016-03-06    0.014043
2016-03-07    0.036014
2016-03-08    0.033296
2016-03-09    0.033090
2016-03-10    0.032184
2016-03-11    0.032575
2016-03-12    0.036920
2016-03-13    0.015670
2016-03-14    0.036549
2016-03-15    0.034284
2016-03-16    0.029610
2016-03-17    0.031628
2016-03-18    0.012911
2016-03-19    0.034778
2016-03-20    0.037887
2016-03-21    0.037373
2016-03-22    0.032987
2016-03-23    0.032225
2016-03-24    0.029342
2016-03-25    0.031607
2016-03-26    0.032204
2016-03-27    0.031092
2016-03-28    0.034860
2016-03-29    0.034099
2016-03-30    0.033687
2016-03-31    0.031834
2016-04-01    0.033687
2016-04-02    0.035478
2016-04-03    0.038608
2016-04-04    0.036487
2016-04-05    0.013096
2016-04-06    0.003171
2016-04-07    0.001400
Name: date_crawled, dtype: float64

In [80]:
(autos['ad_created'].str[:10].
 value_counts(normalize=True, dropna=False).
 sort_index())


2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
2015-12-30    0.000021
2016-01-03    0.000021
2016-01-07    0.000021
2016-01-10    0.000041
2016-01-13    0.000021
2016-01-14    0.000021
2016-01-16    0.000021
2016-01-22    0.000021
2016-01-27    0.000062
2016-01-29    0.000021
2016-02-01    0.000021
2016-02-02    0.000041
2016-02-05    0.000041
2016-02-07    0.000021
2016-02-08    0.000021
2016-02-09    0.000021
2016-02-11    0.000021
2016-02-12    0.000041
2016-02-14    0.000041
2016-02-16    0.000021
2016-02-17    0.000021
2016-02-18    0.000041
2016-02-19    0.000062
2016-02-20    0.000041
2016-02-21    0.000062
                ...   
2016-03-09    0.033151
2016-03-10    0.031895
2016-03-11    0.032904
2016-03-12    0.036755
2016-03-13    0.017008
2016-03-14    0.035190
2016-03-15    0.034016
2016-03-16    0.030125
2016-03-17    0.031278
2016-03-18    0.013590
2016-03-19    0.033687
2016-03-20    0.037949
2016-03-21 

In [81]:
(autos['last_seen'].str[:10].
 value_counts(normalize=True, dropna=False).
 sort_index())

2016-03-05    0.001071
2016-03-06    0.004324
2016-03-07    0.005395
2016-03-08    0.007413
2016-03-09    0.009595
2016-03-10    0.010666
2016-03-11    0.012375
2016-03-12    0.023783
2016-03-13    0.008895
2016-03-14    0.012602
2016-03-15    0.015876
2016-03-16    0.016452
2016-03-17    0.028086
2016-03-18    0.007351
2016-03-19    0.015834
2016-03-20    0.020653
2016-03-21    0.020632
2016-03-22    0.021373
2016-03-23    0.018532
2016-03-24    0.019767
2016-03-25    0.019211
2016-03-26    0.016802
2016-03-27    0.015649
2016-03-28    0.020859
2016-03-29    0.022341
2016-03-30    0.024771
2016-03-31    0.023783
2016-04-01    0.022794
2016-04-02    0.024915
2016-04-03    0.025203
2016-04-04    0.024483
2016-04-05    0.124761
2016-04-06    0.221806
2016-04-07    0.131947
Name: last_seen, dtype: float64

Because a car can't be first registered after the listing was seen, any vehicle with a registration year above 2016 is definitely inaccurate. Determining the earliest valid year is more difficult. Realistically, it could be somewhere in the first few decades of the 1900s.

Let's count the number of listings with cars that fall outside the 1900 - 2016 interval and see if it's safe to remove those rows entirely, or if we need more custom logic.

In [82]:
autos['registration_year'].describe()
# most of registration from 2008
# max year is a mistake 9999

autos['registration_year'].value_counts().sort_index()
# registration year before 1800 seems impossible and 
#after 2800 is not possible

autos = autos[autos['registration_year'].between(1800,2800)]

autos['registration_year'].value_counts(normalize=True).head(20)
# most registrations between 1995 and 2016

2000    0.065008
2005    0.060476
1999    0.059673
2004    0.055677
2003    0.055594
2006    0.054997
2001    0.054297
2002    0.051207
1998    0.048673
2007    0.046902
2008    0.045625
2009    0.042947
1997    0.040187
2011    0.033431
2010    0.032730
2017    0.028673
1996    0.028281
2012    0.026984
1995    0.025274
2016    0.025130
Name: registration_year, dtype: float64

In [83]:
brands = autos["brand"].value_counts(normalize=True)
print(brands)

volkswagen        0.212820
bmw               0.108635
opel              0.108635
mercedes_benz     0.095781
audi              0.085853
ford              0.069663
renault           0.047891
peugeot           0.029455
fiat              0.025995
seat              0.018930
skoda             0.016067
nissan            0.015263
mazda             0.015222
smart             0.014295
citroen           0.014110
toyota            0.012585
hyundai           0.009949
sonstige_autos    0.009640
volvo             0.009043
mini              0.008610
mitsubishi        0.008219
honda             0.007992
kia               0.007106
alfa_romeo        0.006612
porsche           0.005912
suzuki            0.005891
chevrolet         0.005664
chrysler          0.003481
dacia             0.002657
daihatsu          0.002513
jeep              0.002204
subaru            0.002101
land_rover        0.002039
saab              0.001627
daewoo            0.001565
jaguar            0.001524
trabant           0.001380
r

From all the brands volkswagen,bmw and opel are the most popular
If we want to see which brand share 50% of the market

In [84]:
common_brands = brands[brands > 0.05].index
print(common_brands)

Index(['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford'], dtype='object')


In [85]:
# calculate mean prices by brand

brand_mean_prices={}
brand = autos["brand"].unique()

for b in common_brands:
    company = autos[autos['brand'] == b]
    mean_price = company['price'].mean()
    brand_mean_prices[b] = int(mean_price)
    
print(brand_mean_prices)


{'ford': 3728, 'audi': 9212, 'volkswagen': 5332, 'mercedes_benz': 8526, 'opel': 2941, 'bmw': 8261}


We see that form the most common brands:

Audi, BMW and Mercedes Benz are more expensive
Ford and Opel are less expensive
Volkswagen is in between

Comparing prices and mileage

In [86]:
brand_mean_mileage = {}

for b in common_brands:
    company = autos[autos["brand"] == b]
    miles = company["odometer_km"].mean()
    brand_mean_mileage[b] = miles
    
print(brand_mean_mileage)
        


{'ford': 124349.49733885274, 'audi': 129492.56238003839, 'volkswagen': 128943.57336430508, 'mercedes_benz': 130848.3870967742, 'opel': 129452.02882062951, 'bmw': 132682.97307546454}


In [87]:
# convert from ditionary to series
bmp_series = pd.Series(brand_mean_prices) 
#from series to data frame
bmp_df = pd.DataFrame(bmp_series, columns=['mean_price'])

bmm_series = pd.Series(brand_mean_mileage)
bmm_df = pd.DataFrame(bmm_series, columns=['mean_mileage'])

#print(bmp_df)
#print(bmm_df)

bmp_df['mean_mileage']=bmm_df

print(bmp_df)

               mean_price   mean_mileage
audi                 9212  129492.562380
bmw                  8261  132682.973075
ford                 3728  124349.497339
mercedes_benz        8526  130848.387097
opel                 2941  129452.028821
volkswagen           5332  128943.573364


Mean milage does not change with price, opel is considerably cheaper and milage is not very different from the rest

Data cleaning next steps:

See if there are particular keywords in the name column that you can extract as new columns

In [88]:
autos['date_crawled'] = (autos['date_crawled'].str[:10].
                         str.replace("-","").astype(int)
                        )

autos['ad_created'] = (autos['ad_created'].str[:10].
                         str.replace("-","").astype(int)
                        )

autos['last_seen'] = (autos['last_seen'].str[:10].
                         str.replace("-","").astype(int)
                        )

Identify categorical data that uses german words, translate them and map the values to their english counterparts

In [89]:
autos.head()

,date_crawled,name,price,abtest,vehicle_type,registration_year,gearbox,powerPS,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postalCode,last_seen
0,20160326,Peugeot_807_160_NAVTECH_ON_BOARD,5000,control,bus,2004,manuell,158,andere,150000,3,lpg,peugeot,nein,20160326,79588,20160406
1,20160404,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,8500,control,limousine,1997,automatik,286,7er,150000,6,benzin,bmw,nein,20160404,71034,20160406
2,20160326,Volkswagen_Golf_1.6_United,8990,test,limousine,2009,manuell,102,golf,70000,7,benzin,volkswagen,nein,20160326,35394,20160406
3,20160312,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,4350,control,kleinwagen,2007,automatik,71,fortwo,70000,6,benzin,smart,nein,20160312,33729,20160315
4,20160401,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,1350,test,kombi,2003,manuell,0,focus,150000,7,benzin,ford,nein,20160401,39218,20160401


In [90]:
# determine names in german
autos['vehicle_type'].unique()
autos['gearbox'].unique()
autos['unrepaired_damage'].unique()
autos['fuel_type'].unique()
autos['model'].unique()

#replace the names in german for names in english
autos['gearbox'] = autos['gearbox'].replace(['manuell','automatik'],['manual','automatic'])
autos['unrepaired_damage'] = autos['unrepaired_damage'].replace(['nein','ja'],['no','yes'])
autos['fuel_type'] = autos['fuel_type'].replace(['benzin', 'elektro', 'andere'], ['gasoline', 'electric', 'other'])
autos['vehicle_type'] = autos['vehicle_type'].replace(['limousine', 'kleinwagen', 'kombi', 'cabrio', 'andere'], ['sedan', 'small car', 'stationwagen', 'convertible', 'other'])
autos['model'] = autos['model'].replace('andere', 'other')

autos.head()

,date_crawled,name,price,abtest,vehicle_type,registration_year,gearbox,powerPS,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postalCode,last_seen
0,20160326,Peugeot_807_160_NAVTECH_ON_BOARD,5000,control,bus,2004,manual,158,other,150000,3,lpg,peugeot,no,20160326,79588,20160406
1,20160404,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,8500,control,sedan,1997,automatic,286,7er,150000,6,gasoline,bmw,no,20160404,71034,20160406
2,20160326,Volkswagen_Golf_1.6_United,8990,test,sedan,2009,manual,102,golf,70000,7,gasoline,volkswagen,no,20160326,35394,20160406
3,20160312,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,4350,control,small car,2007,automatic,71,fortwo,70000,6,gasoline,smart,no,20160312,33729,20160315
4,20160401,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,1350,test,stationwagen,2003,manual,0,focus,150000,7,gasoline,ford,no,20160401,39218,20160401


Next we will try to find the most common brand/model combinations

In [109]:
# group the dataset by model and brand
model_brand = autos.groupby(["brand","model"])
# find the count and sort descending
brand_model = model_brand["date_crawled"].count().sort_values(ascending=False)
brand_model


brand          model             
volkswagen     golf                  3898
bmw            3er                   2686
volkswagen     polo                  1688
opel           corsa                 1680
               astra                 1410
volkswagen     passat                1383
audi           a4                    1256
mercedes_benz  c_klasse              1161
bmw            5er                   1150
mercedes_benz  e_klasse               977
audi           a3                     866
               a6                     818
ford           focus                  793
               fiesta                 752
volkswagen     transporter            689
renault        twingo                 654
peugeot        2_reihe                610
mercedes_benz  a_klasse               583
smart          fortwo                 572
opel           vectra                 558
bmw            1er                    534
renault        clio                   497
ford           mondeo                 487


Volkswagen Golf is the most common brand/model combination with total 3707 listings, followed by BMW 3ER and Volkswagen Polo

How much cheaper are cars with damage than their non-damaged counterparts?

In [118]:
autos.head()

no_damage = autos[autos['unrepaired_damage'] == 'no']
no_damage_mean = no_damage['price'].mean()
print(no_damage_mean)

yes_damage = autos[autos['unrepaired_damage'] == 'yes']
yes_damage_mean = yes_damage['price'].mean()
print(yes_damage_mean)




7085.720674086217
2221.887609298358


Cars with no damage have a higher mean price than those with no damage